In [1]:
# Clipping ESA land use data to Estonia
# For me personally this part is no longer important, because I chose to store the (clipped) files locally. But in case I need to explain how I got the result, it might be useful to keep this part as well

import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import box

# Path to the local directory containing GeoTIFF files

local_tif_directory = 'C:/Users/egert/dggs_t1-master/ESA_data'

# I used the file list you provided in your working notes to get the maps. But it seems like either ESA or Google changed something, so making a list and then looping through it no longer works
# My best guess is that ESA made these files more difficult to access (on their website it now asks for verification and data such as name, organisation and email)
# Fortunately I have stored these files locally and they are also uploaded to GitHub. It might only make replicating more difficult

# Define the bounding box coordinates for Estonia
minx, miny, maxx, maxy = 21.8375, 57.5246, 28.1945, 59.6750

# Create a shapely geometry using the bounding box coordinates
bbox = box(minx, miny, maxx, maxy)

# Loop through each GeoTIFF file in the local directory
for filename in os.listdir(local_tif_directory):
    if filename.endswith('.tif'):
        tif_path = os.path.join(local_tif_directory, filename)
        
        try:
            # Open the GeoTIFF file from the local path
            with rasterio.open(tif_path) as src:
                # Clip the raster to the bounding box geometry
                clipped_data, clipped_transform = mask(dataset=src, shapes=[bbox], crop=True)

                # Update the metadata (e.g., transform, width, height)
                clipped_meta = src.meta.copy()
                clipped_meta.update({
                    "driver": "GTiff",
                    "height": clipped_data.shape[1],
                    "width": clipped_data.shape[2],
                    "transform": clipped_transform
                })

                # Define the output filename
                output_filename = f'{os.path.splitext(filename)[0]}_clipped.tif'
                output_path = os.path.join(local_tif_directory, output_filename)

                # Write the clipped data to a new GeoTIFF file
                with rasterio.open(output_path, 'w', **clipped_meta) as dst:
                    dst.write(clipped_data)
                    
            # If the processing succeeds, print it with the file name
            print(f'Successfully processed: {output_filename}')
            
        # If the processing fails, print out the path and the error code
        except rasterio.errors.RasterioIOError as e:
            print(f'Error processing {tif_path}: {e}')


Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1_cog_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1_cog_clipped_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1_cog_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1_cog_clipped_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1_cog_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2018-v2.1.1_cog_clipped_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2019-v2.1.1_cog_clipped_clipped.tif
Successfully processed: C3S-LC-L4-LCCS-Map-300m-P1Y-2019-v2.1.1_cog_clipped_clipped_clipped.tif
Successfully processed: ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7_cog_clipped_clipped.tif
Successfully processed: ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7_cog_clipped_clipped_clipped.tif
Successfully processed: ESACCI-LC-L4-LCCS-Map-300m-P1Y-1993-v2.0.7_cog_cli

In [5]:
import os
import geopandas as gpd
import rasterio
from dggrid4py import DGGRIDv7
from rasterstats import zonal_stats
from shapely.geometry import Point, box
import pandas as pd

# Path to the directory containing all land use GeoTIFF files
landuse_dir = r'C:\Users\egert\dggs_t1-master\ESA_data'

# Automatically collect all .tif files from the directory
landuse_rasters = [os.path.join(landuse_dir, f) for f in os.listdir(landuse_dir) if f.endswith('.tif')]

# Path to the output directory
output_dir = r'C:\Users\egert\dggs_t1-master\zonal_stats'

# DGGRID instance (path to dggrid executable)
dggrid_instance = DGGRIDv7(executable=r'C:\ProgramData\miniconda3\envs\dggs_t1\Library\bin\dggrid.exe', 
                           working_dir='.', capture_logs=False, silent=False)

# Copied these two bounding boxes from the last part, in case I decide to remove it.
# Define the bounding box coordinates for Estonia
minx, miny, maxx, maxy = 21.8375, 57.5246, 28.1945, 59.6750

# Create a shapely geometry using the bounding box coordinates
bbox = box(minx, miny, maxx, maxy)

# DGGS configurations: ISEA7H and ISEA3H with different resolutions
dggs_configurations = [
    ('ISEA7H', 5),  # ISEA7H grid at resolution 5
    ('ISEA7H', 8),  # ISEA7H grid at resolution 8
    ('ISEA3H', 7)   # ISEA3H grid at resolution 7
]

# Loop over all DGGS configurations and land use rasters
for dggs_type, resolution in dggs_configurations:
    # Generate DGGRID polygons for Estonia
    gdf_dggs = dggrid_instance.grid_cell_polygons_for_extent(dggs_type, resolution, clip_geom=bbox)
    
    for raster_path in landuse_rasters:
        # Read the land use raster data
        with rasterio.open(raster_path) as src:
            # Perform zonal statistics (mode/majority) for each DGGS polygon
            stats = zonal_stats(gdf_dggs, src.read(1), affine=src.transform, stats=['majority'])
        
        # Add the zonal stats result (majority land use class) to the DGGS GeoDataFrame
        gdf_dggs['landuse_majority'] = [stat['majority'] for stat in stats]
        
        # Extract the year from the raster filename
        year = os.path.basename(raster_path).split('_')[1]  # Adjust if filenames have a different structure
        output_file = os.path.join(output_dir, f'{dggs_type}_{resolution}_landuse_{year}.shp')
        
        # Save the DGGS grid with majority land use to a shapefile
        gdf_dggs.to_file(output_file)
        print(f'Saved {output_file}')
        
    # Create hexagonal points for querying land use at smaller scale
    gdf_points = dggrid_instance.cells_for_geo_points(gdf_dggs, False, dggs_type, resolution + 2)  # Smaller resolution
    
    # Query land use at each point for each year
    for raster_path in landuse_rasters:
        with rasterio.open(raster_path) as src:
            # For each point, get the land use class at that point
            point_landuse = []
            for geom in gdf_points.geometry:
                row, col = src.index(geom.x, geom.y)
                landuse_value = src.read(1)[row, col]
                point_landuse.append(landuse_value)
            
            year = os.path.basename(raster_path).split('_')[1]  # Extract the year
            gdf_points[f'landuse_{year}'] = point_landuse
        
        # Save the point queries to a file
        point_output_file = os.path.join(output_dir, f'{dggs_type}_{resolution}_landuse_points_{year}.shp')
        gdf_points.to_file(point_output_file)
        print(f'Saved point query: {point_output_file}')

# Combine the zonal stats and point queries into a single table for analysis
combined_data = pd.DataFrame()

for dggs_type, resolution in dggs_configurations:
    for raster_path in landuse_rasters:
        year = os.path.basename(raster_path).split('_')[1]
        zonal_file = os.path.join(output_dir, f'{dggs_type}_{resolution}_landuse_{year}.shp')
        point_file = os.path.join(output_dir, f'{dggs_type}_{resolution}_landuse_points_{year}.shp')
        
        # Read both files
        gdf_zonal = gpd.read_file(zonal_file)
        gdf_point = gpd.read_file(point_file)
        
        # Combine land use data from polygons and points
        combined_df = gdf_zonal[['cell_id', 'landuse_majority']].merge(
            gdf_point[['cell_id', f'landuse_{year}']], on='cell_id')
        
        # Add to the combined data
        combined_data = pd.concat([combined_data, combined_df])

# Save the final combined data
combined_data.to_csv(os.path.join(output_dir, 'combined_landuse_stats.csv'), index=False)
print('Saved combined land use statistics.')



** executing DGGRID version 7.8 **
type sizes: big int: 64 bits / big double: 64 bits

** loading meta file metafile_fa5b82ae-5b62-4e86-955c-26b5eb17ffb6...
* using parameter values:
dggrid_operation GENERATE_GRID (user set)
dggs_type ISEA7H (user set)
dggs_topology HEXAGON (user set)
dggs_proj ISEA (user set)
dggs_aperture_type PURE (user set)
dggs_aperture 7 (user set)
proj_datum WGS84_AUTHALIC_SPHERE (default)
dggs_orient_specify_type SPECIFIED (user set)
dggs_num_placements 1 (user set)
dggs_orient_output_file_name grid.meta (default)
dggs_vert0_lon 11.25 (user set)
dggs_vert0_lat 58.2825 (user set)
dggs_vert0_azimuth 0 (user set)
dggs_res_specify_type SPECIFIED (user set)
dggs_res_spec 5 (user set)
rng_type RAND (default)
geodetic_densify 0 (default)
clip_subset_type GDAL (user set)
clip_cell_addresses  (default)
clip_cell_res 1 (default)
clip_cell_densification 1 (default)
clip_region_files C:\Users\egert\dggs_t1-master\temp_clip_32c5a14c-3cdb-41be-ace7-44a2c50b54f4.geojson (user

C:\Users\egert\AppData\Local\Temp\ipykernel_20972\2187059024.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_dggs.to_file(output_file)


Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp


C:\Users\egert\AppData\Local\Temp\ipykernel_20972\2187059024.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_dggs.to_file(output_file)


Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp


...

Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp
Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp


C:\Users\egert\AppData\Local\Temp\ipykernel_20972\2187059024.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_dggs.to_file(output_file)
C:\Users\egert\AppData\Local\Temp\ipykernel_20972\2187059024.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_dggs.to_file(output_file)


Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp
Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp
Saved C:\Users\egert\dggs_t1-master\zonal_stats\ISEA7H_5_landuse_cog.shp


C:\Users\egert\AppData\Local\Temp\ipykernel_20972\2187059024.py:55: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_dggs.to_file(output_file)


ValueError: x attribute access only provided for Point geometries